In [ ]:
import MySQLdb
db = MySQLdb.connect("localhost", "root", "root", "house")
db.set_character_set('utf8')
cursor = db.cursor()
sql = "select link from link_list where status = 1 limit 1"

cursor.execute(sql)
results = cursor.fetchall()
print results
def printa():
    print 1
print results
printa()

def readSql(sql):
    cursor.execute(sql)
    results = cursor.fetchall()
    return results

(('http://bj.lianjia.com/chengjiao/p1a1l1sf1lc1f1pg1',),)
(('http://bj.lianjia.com/chengjiao/p1a1l1sf1lc1f1pg1',),)
1


目前所有表:

In [13]:
sql = "show tables"
for res in readSql(sql):
    print res[0]

link_list
ods_house_da
ods_house_new
ori_house_da


In [14]:
import re
region_set = set()
area_set = set()
area_region_dict = {}
sql = "select href, title from ori_house_da limit 10000000"
p = re.compile('\D+')
countY = 0
countN = 0
for res in readSql(sql):
    tmp = res[0].replace("http://bj.lianjia.com/chengjiao/", "").replace(".html", "")
    try:
        region = p.match(tmp).group(0)
        region_set.add(region)
        countY += 1
        area = res[1].split(" ")[0]
        area_set.add(area)
        area_region_dict[area] = region
    except:
        #print res
        countN += 1
print region_set
print "小区数量：" + str(len(area_set))
cnt = 0
for i in area_set:
    if cnt < 10:
        cnt += 1
        print i
print "Already reginonID handle: " + str(countY)
print "Cannot reginonID handle: " + str(countN)

set(['BJCP', 'BJMT', 'BJHD', 'BJXW', 'BJCW', 'BJDC', 'BJCY', 'LFYJ', 'BJSJ', 'LFYj', 'lFYJ', 'BJFS', 'LFAC', 'BJXC', 'BJFT', 'BJYZ', 'BJSY', 'BJDX', 'BJTJ'])
小区数量：4356
北街家园六区
威毕欧溪谷
景馨园
广安门外车站东街
永居东里
海户西里33号院
宝盛里
城市印象
竹园
玉泉路16号院
Already reginonID handle: 99888
Cannot reginonID handle: 19876


In [15]:
if "" in area_region_dict:
    print "",area_region_dict[""]

In [16]:
from random import randint
import re
sql = "select href, title, id, dealDate, totalPrice, dealHouseTxt from ori_house_da \
          where totalPrice != '' and dealDate != '' and title != ''"
print sql
p = re.compile('\D+')
countY = 0
countN = 0
for res in readSql(sql):
    href = res[0]
    tmp = href.replace("http://bj.lianjia.com/chengjiao/", "").replace(".html", "")
    title = res[1]
    id = res[2]
    dealDate = res[3]
    totalPrice = res[4]
    dealHouseTxt = res[5]
    try:
        countY += 1
        area = title.split(" ")[0]
        struct = title.split(" ")[1]
        room = struct.replace("室","").replace("厅","")
        square = title.split(" ")[2].replace("平米","")
        region = p.match(tmp).group(0)
    except:
        if area in area_region_dict.keys():
            region = area_region_dict[area]
            countY += 1
        else:
            countN += 1
            continue
    if len(dealDate.split('.')) == 2:
        dealDate = dealDate + "." + str(randint(10,28))
    five = 0
    if "满五年" in dealHouseTxt:
        five = 1
    subway = 0
    if "号线" in dealHouseTxt:
        subway = 1
    sql = "INSERT into ods_house_new(id, region, area, struct, room, square, dealDate \
              ,dealHouseTxt, five, subway, totalPrice) VALUES ('%d', '%s', '%s', '%s', '%s',  \
              '%s', '%s', '%s', '%d', '%d' ,'%s')" % (id, region, area, struct, room, square, \
                dealDate, dealHouseTxt, five, subway, totalPrice)
    if countY < 10:
        print id, region, area, struct, room, square, dealDate, dealHouseTxt, five, subway
        print sql
    cursor.execute(sql)
db.commit()
print "Already reginonID handle: " + str(countY)
print "Cannot reginonID handle: " + str(countN)

select href, title, id, dealDate, totalPrice, dealHouseTxt from ori_house_da           where totalPrice != '' and dealDate != '' and title != ''
-9222965325687740466 BJFT 太平桥西里 1室1厅 11 48.67 2015.09.27 ###房本满五年###距9号线六里桥东340米 1 1
INSERT into ods_house_new(id, region, area, struct, room, square, dealDate               ,dealHouseTxt, five, subway, totalPrice) VALUES ('-9222965325687740466', 'BJFT', '太平桥西里', '1室1厅', '11',                '48.67', '2015.09.27', '###房本满五年###距9号线六里桥东340米', '1', '1' ,'154')
-9222399307971189537 BJCY 和平街十五区1号院 1室1厅 11 47.08 2014.12.05 ###房本满五年###距13号线光熙门247米 1 1
INSERT into ods_house_new(id, region, area, struct, room, square, dealDate               ,dealHouseTxt, five, subway, totalPrice) VALUES ('-9222399307971189537', 'BJCY', '和平街十五区1号院', '1室1厅', '11',                '47.08', '2014.12.05', '###房本满五年###距13号线光熙门247米', '1', '1' ,'206')
-9221846324689872064 BJDC 绿景苑 2室1厅 21 60.93 2011.09.29 ###房本满五年###距7号线广渠门内707米 1 1
INSERT into ods_house_new(id, region, area, 

In [10]:
db.close()